## Passive Seismic Metadata Standards (Geoscience Australia) v(0.2)

This document describes how corrections for GPS clock-errors and station-orientations are encapsulated in StationXML files.  

### 1. QA/QC Procedure

[HiPerSeis](https://github.com/GeoscienceAustralia/hiperseis) implements elaborate workflows for computing corrections for:

1. [GPS clock errors](https://github.com/GeoscienceAustralia/hiperseis/tree/develop/seismic/gps_corrections)
 
 This workflow outputs simple csv files for each station processed, as follows:
 ```
 net,sta,loc,comp,date,clock_correction
 OA,BM26,,HHZ,2019-05-25T00:00:00,0.040261028
 OA,BM26,,HHZ,2019-05-26T00:00:00,0.269642617
 OA,BM26,,HHZ,2019-05-27T00:00:00,0.518772089
 OA,BM26,,HHZ,2019-05-28T00:00:00,0.787649446
 OA,BM26,,HHZ,2019-05-29T00:00:00,1.076274687
 ...
  ```
 Each row above lists the daily clock-correction (in seconds) for a given station. 
 
 
2. [Station-orientation errors](https://github.com/GeoscienceAustralia/hiperseis/tree/develop/seismic#orientation-analysis)

 This workflow generates orientation correction estimates based on two separate 
 methods: (i) Receiver function ([Wilde-Piórko et al. 2017](https://doi.org/10.1007/s10950-017-9640-x)), (ii) Surface wave polarization ([Doran and Lakse, 2017](https://doi.org/10.1785/0120160165))

 Correction estimates are output in Json format as follows:
 ```json
 {
    "rf": {
        "OA.BK24.": {
            "date_range": [
                "2019-05-30T15:45:33.505000Z",
                "2020-09-13T11:58:30.490000Z"
            ],
            "azimuth_correction": -4.0
        }
        ...
        ...
    }
    "swp": {
        "OA.BK24.": {
            "date_range": [
                "2019-05-26T07:40:04.360000Z",
                "2020-09-11T11:35:53.700000Z"
            ],
            "azimuth_correction": 1.3485540759336345,
            "uncertainty": 3.6205002297639233
        }
        ...
        ...
    }
}
``` 
 Correction estimates from the surface wave polarization method have associated uncertainties, which are currently unavailable for those from the receiver function method.

### 2. Embedding QC/QC Metadata into StationXML Files

Obspy [inventory utilities](https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.html) are used to embed the corrections described above within StationXML files as [extra station metadata](https://docs.obspy.org/tutorial/code_snippets/stationxml_custom_tags.html).

Once these QA/QC metadata are embedded, a typical StationXML file looks like as follows:

```xml
<Network code="OA" 
         startDate="2017-09-11T00:00:36" 
         endDate="2018-11-28T23:06:20">
    <Station code="BY22" 
             startDate="2017-10-01T13:58:58" 
             endDate="2018-11-24T21:13:47">
      <GeoscienceAustralia:clock_corrections>{"0M": [{"2018-07-28T00:00:00 - 2018-07-29T00:00:00": 0.432706767}, {"2018-07-29T00:00:00 - 2018-07-30T00:00:00": 0.768295739}, {"2018-07-30T00:00:00 - 2018-07-31T00:00:00": 1.124155978}, {"2018-07-31T00:00:00 - 2018-08-01T00:00:00": 1.500287483}, {"2018-08-01T00:00:00 - 2018-08-02T00:00:00": 1.896690255}, {"2018-08-02T00:00:00 - 2018-08-03T00:00:00": 2.313364293}, {"2018-08-03T00:00:00 - 2018-08-04T00:00:00": 2.750309598}, {"2018-08-04T00:00:00 - 2018-08-05T00:00:00": 3.207526168}, {"2018-08-05T00:00:00 - 2018-08-06T00:00:00": 3.685014006}, {"2018-08-06T00:00:00 - 2018-08-07T00:00:00": 4.182773109}, {"2018-08-07T00:00:00 - 2018-08-08T00:00:00": 4.700803479}, {"2018-08-08T00:00:00 - 2018-08-09T00:00:00": 5.239105116}, {"2018-08-09T00:00:00 - 2018-08-10T00:00:00": 5.797678019}, {"2018-08-10T00:00:00 - 2018-08-11T00:00:00": 6.376522188}, {"2018-08-11T00:00:00 - 2018-08-12T00:00:00": 6.975637623}, {"2018-08-12T00:00:00 - 2018-08-13T00:00:00": 7.595024326}, {"2018-08-13T00:00:00 - 2018-08-14T00:00:00": 8.234682294}, {"2018-08-14T00:00:00 - 2018-08-15T00:00:00": 8.894611529}, {"2018-08-15T00:00:00 - 2018-08-16T00:00:00": 9.57481203}]}
      </GeoscienceAustralia:clock_corrections>
      
      <GeoscienceAustralia:rf_orientation_corrections>{"0M": {"2017-10-06T08:08:02 - 2018-11-22T16:21:07": 1.0}}         
      </GeoscienceAustralia:rf_orientation_corrections>

      <GeoscienceAustralia:swp_orientation_corrections>{"0M": {"2017-10-03T06:52:09 - 2018-11-23T14:51:59": [2.0, 1.9]}}
      </GeoscienceAustralia:swp_orientation_corrections>
      
...
...
```

For each station, `clock_corrections` contain lists of dictionaries with timespan as the key and clock-correction as the value in seconds, for each location code. Similarly, `rf_orientation_corrections` and `swp_orientation_corrections` contain orientation correction estimates. 

### 3. Extracting QA/QC Metadata

QA/QC metadata can be extracted through the Obspy [Inventory](https://docs.obspy.org/tutorial/code_snippets/stationxml_custom_tags.html) interface as follows: 

In [6]:
from obspy import Inventory
from obspy.core.inventory import Network, read_inventory

#inv = read_inventory('/g/data/ha3/rakib/seismic/pst/inv_meta/test.xml')
inv = read_inventory('OA.xml')
sta = inv.networks[0].stations[-1]

# Extract corrections from the inventory 
if(hasattr(sta.extra, 'clock_corrections')):
    print('Clock corrections for station {}:\n'.format(sta.code))
    print(sta.extra.clock_corrections)
# end if

if(hasattr(sta.extra, 'rf_orientation_corrections')):
    print('\nRF Orientation correction for station {}:\n'.format(sta.code))
    print(sta.extra.rf_orientation_corrections)
# end if

if(hasattr(sta.extra, 'swp_orientation_corrections')):
    print('\nSWP Orientation correction and uncertainty ' \
          'for station {}:\n'.format(sta.code))
    print(sta.extra.swp_orientation_corrections)
# end if

Clock corrections for station CD22:

AttribDict({'namespace': 'https://github.com/GeoscienceAustralia/hiperseis', 'value': '{"0M": [{"2017-12-02T00:00:00 - 2017-12-03T00:00:00": -1.25}, {"2017-12-03T00:00:00 - 2017-12-04T00:00:00": -1.25}, {"2017-12-04T00:00:00 - 2017-12-05T00:00:00": -0.75}, {"2017-12-18T00:00:00 - 2017-12-19T00:00:00": 0.25}, {"2017-12-19T00:00:00 - 2017-12-20T00:00:00": 0.5}, {"2017-12-24T00:00:00 - 2017-12-25T00:00:00": 0.492965368}, {"2017-12-25T00:00:00 - 2017-12-26T00:00:00": 0.292207792}, {"2017-12-26T00:00:00 - 2017-12-27T00:00:00": 0.394480519}, {"2017-12-27T00:00:00 - 2017-12-28T00:00:00": 0.390692641}, {"2017-12-28T00:00:00 - 2017-12-29T00:00:00": 0.394480519}, {"2017-12-29T00:00:00 - 2017-12-30T00:00:00": 0.542207792}, {"2017-12-30T00:00:00 - 2017-12-31T00:00:00": 0.492965368}, {"2018-01-01T00:00:00 - 2018-01-02T00:00:00": 0.5}, {"2018-01-02T00:00:00 - 2018-01-03T00:00:00": 0.5}, {"2018-01-12T00:00:00 - 2018-01-13T00:00:00": 0.942857143}, {"2018-01-13T00:0

### 4. Metadata Extractor Script

In addition to accessing metadata programmatically above, a script [export_corrections.py](https://github.com/GeoscienceAustralia/hiperseis/blob/develop/seismic/inventory/export_corrections.py) is provided to extract the metadata into csv files.

#### Requirements

python3, obspy (version 1.1.0), pandas

#### Launch Exporter

The meta-data exporter is launched as follows:

`python export_corrections.py OA.xml meta`

The inputs are: (i) a StationXML file with embedded metadata (ii) output-file base-name

Upon successful completion, two csv files should be produced:

 1. `meta.clock_corrections.csv`, with typical content as follows:
 ```
# GPS clock-corrections grouped by network, station and location
Network,Station,Location,Start-time,End-time,Correction_in_seconds
OA,BS25,0M,2018-04-16T00:00:00.000000,2018-04-17T00:00:00.000000,-0.916666667
OA,BS25,0M,2018-04-17T00:00:00.000000,2018-04-18T00:00:00.000000,-0.886904762
 ...
 ...
 ```
 2. `meta.orientation_corrections.csv`, with typical content as follows:
```
# Orientation corrections are derived from two separate methods: (i) Receiver Function (RF) (ii) Surface-wave Polarization (SWP). Only the latter method provides uncertainty estimates.
Network,Station,Location,Method,Start-time,End-time,Azimuth_correction_in_degrees,Uncertainty±
OA,BS24,0M,RF,2017-10-06T08:08:12.755000,2018-08-17T22:17:31.755000,2.0,
OA,BS25,,RF,2019-12-04T20:15:16.408000,2021-03-19T19:02:55.508000,-1.0,
...
...
```
